In [1]:
import cv2 as cv
import numpy as np
import tensorflow as tf

from helpers.utils import show_image, normalize, preprocess_image, detect_sudoku, extract_cells, sobel_gradients, get_binary_labels, get_digit_labels

In [2]:
def isDelimiter(line, threshold):
    mean = line.mean()
    if mean < threshold:
        return True
    return False

def extract_bold_lines(img):
    cell_width = 500 / 9  # each cell takes around 55.5 px
    padding = 5  # px

    lines_coordinates = []
    for i in range(9):
        Ya = i * cell_width
        Yb = (i+1) * cell_width
        Xa, Xb = 0, 0
        for j in range(8):
            Xa += cell_width
            Xb += cell_width
            line = [(int(Xa), int(Ya)), (int(Xa), int(Yb))]
            lines_coordinates.append(line)

    # now we have the coordinates of each small line

    # we find the border value between the two categories (thin and bold line)
    # by computing the mean value of each line, storing them into an array
    # sorting the array and finding the biggest consecutive difference. Once we
    # found that, we set the threshold as the mean of the two elements that
    # determined the biggest difference

    means = []

    # iterate through all 144 lines (72 horizontal and 72 vertical)
    for i in range(9):
        for j in range(8):
            line = lines_coordinates[i * 8 + j]
            Xa, Ya = line[0]
            Xb, Yb = line[1]
            vertical_line = img[Ya: Yb, Xa - padding: Xa + padding].copy()
            horizontal_line = img[Xa - padding: Xa + padding, Ya: Yb].copy()
            v_mean = vertical_line.mean()
            h_mean = horizontal_line.mean() 
            means.append(v_mean)
            means.append(h_mean)

    sorted_means = sorted(means, reverse=True) 
    index = 0
    max_dif = 0
    for i in range(len(sorted_means) - 1):
        dif = sorted_means[i] - sorted_means[i+1]
        if dif > max_dif:
            max_dif = dif
            index = i
    threshold = (sorted_means[index] + sorted_means[index+1]) / 2

    # now iterate through the lines, check if it's bold or thin
    # and store the answer into the following 2 matrices
    vertical_lines_matrix = np.zeros((9, 8))
    horizontal_lines_matrix = np.zeros((8, 9))
    for i in range(9):
        for j in range(8):
            line = lines_coordinates[i * 8 + j]
            Xa, Ya = line[0]
            Xb, Yb = line[1]
            vertical_line = img[Ya: Yb, Xa - padding: Xa + padding].copy()
            if (isDelimiter(vertical_line, threshold)):
                vertical_lines_matrix[i][j] = 1
            # show_image("", vertical_line)
            horizontal_line = img[Xa - padding: Xa + padding, Ya: Yb].copy()
            if (isDelimiter(horizontal_line, threshold)):
                horizontal_lines_matrix[j][i] = 1
            # show_image("", horizontal_line)

    # print(vertical_lines_matrix)
    # print(horizontal_lines_matrix)
    return vertical_lines_matrix, horizontal_lines_matrix

In [3]:
from collections import deque as queue

def possible_move(old, new, horizontal_lines, vertical_lines):
    old_i, old_j = old
    new_i, new_j = new

    # first check if the move is in bounds
    if new_i < 0 or new_i > 8 or new_j < 0 or new_j > 8:
        return False

    # check if there is a line between the two cells:
    if old_i == new_i:  # horizontal move => vertical lines
        return not(vertical_lines[new_i][min(old_j, new_j)])
    else:  # vertical move => horizontal lines
        return not(horizontal_lines[min(old_i, new_i)][new_j])


def fill_region(start_i, start_j, region_number, regions, visited, h_lines, v_lines):
    # performs a bfs starting from (start_i, start_j) and
    # fills the reached cells with the value of region_number
    q = queue()
    q.append((start_i, start_j))
    visited[start_i][start_j] = 1
    while len(q):
        cell = q.popleft()
        x, y = cell
        regions[x][y] = region_number

        moves = [[-1, 0], [1, 0], [0, -1], [0, 1]]
        for move in moves:
            new_i = x + move[0]
            new_j = y + move[1]
            if possible_move((x, y), (new_i, new_j), h_lines, v_lines) and visited[new_i][new_j] == 0:
                q.append((new_i, new_j))
                visited[new_i][new_j] = 1


def fill_regions(regions, visited, h_lines, v_lines):
    # finds all the regions by performing a bfs starting from each
    # unvisited cell
    region_number = 1
    for i in range(9):
        for j in range(9):
            if not(visited[i][j]):
                fill_region(i, j, region_number, regions,
                            visited, h_lines, v_lines)
                region_number += 1


def extract_regions(img):
    vertical_lines, horizontal_lines = extract_bold_lines(img)

    # print(vertical_lines)
    # print(horizontal_lines)

    regions = np.full((9, 9), -1)
    visited = np.zeros((9, 9))

    fill_regions(regions, visited, horizontal_lines, vertical_lines)

    return regions

In [21]:
def extract_information(img):
    img = detect_sudoku(img)  # varying size, colored
    img = cv.resize(img, (500, 500))  # fixed size
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)  # fixed size, grayscale
    img = normalize(img)
    regions = extract_regions(img)
    # print(regions)
    # [[1 2 2 2 2 2 2 2 2]
    #  [1 1 1 3 3 3 2 4 4]
    #  [1 3 3 3 5 5 5 4 4]
    #  [1 1 6 3 3 3 5 4 4]
    #  [1 1 6 6 6 5 5 4 4]
    #  [6 6 6 6 7 5 5 8 4]
    #  [9 9 7 6 7 7 5 8 8]
    #  [9 9 7 7 7 7 8 8 8]
    #  [9 9 9 9 9 7 8 8 8]]
    
    binary_labels = get_binary_labels(img)
    
    model = tf.keras.models.load_model('saved_model/model.h5')
    
    digit_labels = get_digit_labels(img, model)

    return regions, binary_labels, digit_labels


In [5]:
def get_results(input_dir, output_dir, number_of_samples):
    for i in range(1, number_of_samples + 1):
        if i < 10:
            img = cv.imread(
                f"{input_dir}/0{i}.jpg")
        else:
            img = cv.imread(
                f"{input_dir}/{i}.jpg")

        regions, binary_labels, digit_labels = extract_information(img)
        binary_labels = np.array(binary_labels)
        binary_labels = np.reshape(binary_labels, (9, 9))
        digit_labels = np.array(digit_labels)
        digit_labels = np.reshape(digit_labels, (9, 9))
        
        file = open(f'{output_dir}/jigsaw/{i}_predicted.txt', 'w')
        for j in range(9):
            for k in range(9):
                if binary_labels[j][k] == 0:
                    char = 'o'
                else:
                    char = 'x'
                file.write(str(regions[j][k]))
                file.write(char)
            if j != 8:
                file.write('\n')
        file.close()

        file = open(f'{output_dir}/jigsaw/{i}_bonus_predicted.txt', 'w')
        for j in range(9):
            for k in range(9):
                char = str(digit_labels[j][k])
                if char == '0':
                    char = 'o'
                file.write(str(regions[j][k]))
                file.write(char)
            if j != 8:
                file.write('\n')
        file.close()


In [20]:
input_dir = 'datasets/antrenare/jigsaw'
output_dir = 'results'
number_of_samples = 40  # number of input images
get_results(input_dir, output_dir, number_of_samples)